# Neural Networks for Statisticians

## 0. Introduction

Introductions to neural networks are typically presented for people coming from a background in computer science and/or programming, but rarely for people coming from statistics. A statistician hoping to learn about this exciting field frequently encounters a few hurdles:
- Terminological differences may obscure connections between neural networks and standard statistical models
- The machine learning community frequently emphasizes things that statisticians tend to ignore (distributed computing, optimization), while ignoring things that statisticians tend to emphasize (interval estimators, hypothesis testing)
- The machine learning community uses different computational tools than statisticians
- The study of neural networks is an extremely fast-moving field; important new advances appear almost every month

This guide is "for statisticians" in the sense that I assume you know about MLEs, GLMs, etc., and I will motivate neural network practice from that angle. I will **bold** terms that you may not have seen before, or which have a different meaning in neural network-land than they do in statistics ("bias" is the most conspicuous example). I will assume that you have written code before. As long as you know the basics of programming, you should be able to follow along with my code, even if you're not a Python user. Reasons for why Python is the language of choice rather than more "statistical" langauges like R or SAS will be given when we discuss implementing these models.

I will avoid the term "deep learning" here simply because I don't find it necessary to distinguish it from "the study of neural networks" for our purposes here. Also, in my experience, statisticians tend to dismiss it and other newer data-related terms ("data science", "big data", or even "machine learning" [which has actually been around since the 50's]) as contentless buzzwords. I disagree, but we can discuss that some other time.

With all this said, it is worth emphasizing that understanding neural networks requires thinking at a *higher level of abstraction* compared to standard statistical modeling. You won't get very far using a computer for practical purposes if you restrict yourself to thinking in terms of bits and transistors. In the same way, even though (most) neural networks are ultimately statistical models, they are frequently conceptualized in terms of higher-level concepts.

### Required Packages

All required imports to run examples are in this cell; if you can run these, you're good to go:

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

from struct import unpack
import gzip

%matplotlib inline
sns.set()

Using TensorFlow backend.


### Example Dataset

The dataset in examples during this tutorial is [Fashion-MNIST](https://arxiv.org/abs/1708.07747), which consists of 70,000 28-by-28 grayscale images of clothing items, together with which category each article falls in. There are ten categories of object, so the category codes (that is, the $y$-values) range from 0 to 9. More information about the dataset, including what type of clothing each code corresponds to, can be found at the [GitHub repo](https://github.com/zalandoresearch/fashion-mnist) for this dataset.

Based on the designated training set of 60,000 images, our goal is to make good classifications of the remaining 10,000 articles of clothing. Even for such small images and so few categories, this is a challenging task.

I have not included this dataset in this repo. To follow along with the code examples presented here, do the following:
1. Create a directory named `Data` just below this repo's root directory (so "/some/path/here/Notebooks/Data/")
2. Go to the repo for the dataset (the second link above)
3. Download the four `.gz` files from the `README.md` page (train and test set inputs and labels)
4. Move those four files into the Data folder you just created

This process will not be required when the next version of Keras comes out, at which point the data can be directly downloaded through an interface provided by that package.

## 1. Standard Models Rewritten as Neural Nets

I believe that the best place to begin starting to understanding neural networks is to see them from the generalized linear model (GLM) point of view. Simple network architectures can be directly related to the three models presented here; more advanced architectures can then be built on top of those.

Note that despite being written as graphs, it is best to view these models as having *no connection* with either Bayesian networks or decision tree models. This isn't actually true, but for now, pretend that it is.

### Linear Regression

Here is a linear function of $x_1, \ldots, x_p$, written as a neural network:

![Linear Regression](../pics/Linear-Regression.png)

This graph is read from bottom to top. There are two layers in this network: an **input layer** and an **output layer**. Each arrow corresponds to a **weight** $w_j$. The line in the output neuron represents the identity function; it is the **activation function** for that neuron. Input neurons do not have activation functions; they exist solely to pass along their values $x_1$, $x_2$, etc. to the next layer. When interpreting these sorts of diagrams, **bias** terms are generally not explicitly represented in the graph; it is assumed that the bias term is added in before the activation function is evaluated.

So the value of the output neuron is therefore:
$$ \operatorname{identity} \left(b + \sum_{j = 1}^p w_j x_j \right) = b + \sum_{j = 1}^p w_j x_j $$
where $b$ is the bias term and $\mathbf{w}$ is the weight vector. Incoming connections to the output neuron are summed together, a bias term is added, and the activation function is applied.

Now suppose for each $y_i$ value in my training set, I am using this function to make a prediction:
$$ y^*_i = f(\mathbf{x}^{(i)}; \mathbf{w}, b) = b + \sum_{j = 1}^p w_j x^{(i)}_j $$
I want to choose the values of $\mathbf{w}$ and $b$ in order to make my predictions as good as possible, which means *minimizing the badness of my predictions*. Badness is quantified by a **loss function**, which you can choose depending on the problem. A loss function $L(y, y^*)$, for a true $y$ and a corresponding prediction $y^*$, must satisfy the following qualities:
- If $y = y^*$, $L(y, y^*) = 0$ (if your prediction is exactly correct, its badness is 0)
- $\forall y, y^*$, $L(y, y^*) \geq 0$ (there is no such thing as negative badness)

One such function is the *squared-error loss function* $L(y, y^*) = (y - y^*)^2$. Choosing $\mathbf{w}, b$ in order to minimize the average loss incurred by our predicted values (that is, **learning** the weights and bias) is then:
$$ \underset{\mathbf{w}, b}{\operatorname{argmin}} \frac{1}{n} \sum_{i = 1}^n \left[ y_i - \left( b + \sum_{j = 1}^p w_j x_j \right) \right] ^2 $$
where $n$ is the number of examples in your **training set**. This will give you the same weight and bias values as if you had assumed *iid* normal errors and decided to fit the model using maximum likelihood.

### Logistic Regression

![Logistic Regression](../pics/Logistic-Regression.png)

We can do the exact same thing for logistic regression by replacing the identity function in the output neuron with the logistic sigmoid function $\sigma(x) = \frac{1}{1 + e^{-x}}$. It should be clear that after applying the same operations as before, but with $\sigma(x)$ instead of $\operatorname{identity}(x)$, we will get the expectation function for a logistic regression. This is simply because the sigmoid function is the inverse of the typical logit link function.

Obviously the likelihood for a logistic regression model is not equivalent to that of a linear regression model, even after accounting for the different (inverse) link function. When fitting a logistic regression with maximum likelihood, we want:
$$\underset{\mathbf{\beta}}{\operatorname{argmax}} \sum_{i = 1}^n \left[ y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right] =$$
$$\underset{\mathbf{\beta}}{\operatorname{argmax}} \sum_{i = 1}^n \left[ y_i \log \left(\sigma \left[ b + \sum_{j = 1}^p w_j x^{(i)}_j \right] \right) + (1 - y_i) \log \left(1 - \sigma \left[ b + \sum_{j = 1}^p w_j x^{(i)}_j \right] \right) \right]$$

$\sigma \left( b + \sum_{j = 1}^p w_j x_j \right)$ is the value generated by the network, so maximizing the log-likelihood here corresponds to minimizing the average **binary cross-entropy loss**, where "badness" is measured by the function $L(y, y^*) = -y \log(y^*) - (1 - y) \log(1 - y^*)$.

Logistic regression, and neural networks with sigmoid output activations more generally, are frequently used for the purpose of **binary classification**: determining whether observations belong to one of two classes. If the output value is greater than 0.5, the observation would be classified as being in "group 1"; otherwise it will be classified as being in "group 0". This 0.5 threshold may be moved depending on the specific problem being solved.

Note that there is no reason we couldn't use a network with a sigmoid output function and minimize it with respect to squared-error loss, or use the identity function with binary cross-entropy loss. In general, we can use any univariate function we want in the top layer, and minimize the weights with respect to any loss function we want. That said, the optimization problem for a neural network does frequently end up being nothing more than maximum likelihood estimation, under some assumption about the conditional distribution of the $y$'s given the $x$'s.

### Categorical Regression

Generalized linear models with a categorical random component seem to appear less frequently in statistics than in machine learning, so here I will go through the full explanation of this model before connecting it to neural networks.

Consider a design matrix $\mathbf{X}$ and a response matrix $Y$, where the $i^\text{th}$ row of $Y$ is a $k$-dimensional **one-hot** vector (a single entry is 1, the others are 0). This means that the $i^\text{th}$ training example belongs to the $j^\text{th}$ class, where $j \in 1, \ldots, k$, as specified by the location of the 1 in that row.

Consider the "multinomial distribution with $n = 1$", otherwise known as the **categorical distribution** or the **multinoulli distribution**. We will say $\mathbf{y}_i | \mathbf{x}^{(i)} \sim \text{Categorical}(\mathbf{p})$. As with any other GLM, we model the mean of the response distribution ($\mathbf{p}$) as an inverse link function of a linear predictor of the elements of $\mathbf{x}^{(i)}$.

In this case, the inverse link function we use is the [softmax function](https://en.wikipedia.org/wiki/Softmax_function), which is the $k$-dimensional analogue of the logistic function:
$$\operatorname{softmax}(\mathbf{z}) = \frac{e^{\mathbf{z}}}{\sum_{j = 1}^k e^{z_j}}$$
The exponentiation in the numerator is a vectorized operation; each member of $\mathbf{z}$ is exponentiated and the output is a $k$-dimensional vector. Note that if one element of $\mathbf{z}$ is much larger than the others, the output of the softmax function will be approximately 1 at that index, and approximately 0 at the others, just like the vector-valued (hard) max function. Also note that the sum of the $k$ elements of this function's output vector will always be 1. It is therefore appropriate to use this to model the mean of a categorical distribution.

We need a $k$-dimensional linear predictor of $\mathbf{x}_i$ for this model, so we will have a parameter matrix:
$$\mathbf{p}_i = \operatorname{softmax} \left( \mathbf{b} + \mathbf{W} \mathbf{x}^{(i)} \right)$$
where $\mathbf{b}$ is a $k$-dimensional vector and $\mathbf{W}$ is a $k \times p$-dimensional matrix of weights.

So the likelihood function for this model is:
\begin{eqnarray*}
L(\mathbf{\theta}) &=& \prod_{i = 1}^n \prod_{j = 1}^k p_{ij}^{y_{ij}}\\
\ell(\mathbf{\theta}) &=& \sum_{i = 1}^n \sum_{j = 1}^k {y_{ij}} \log(p_{ij})\\
&=& \sum_{i = 1}^n \mathbf{y}_i^T \log(\mathbf{p}_i)\\
&=& \sum_{i = 1}^n \mathbf{y}_i^T \log \left[ \operatorname{softmax} \left( \mathbf{b} + \mathbf{W} \mathbf{x}^{(i)} \right) \right]
\end{eqnarray*}
where the logarithms in the final two lines are understood to be vectorized across their arguments.

#### Categorical Regression as a Neural Network

![Categorical Regression](../pics/Categorical-Regression.png)

Here I've drawn a 3-class categorical regression as a neural network. $\sigma$ is occassionally used to refer to the softmax function as well as the logistic function, so I use $\sigma_i$ to denote its $i^\text{th}$ component. Unlike most neural networks, the neurons in the output layer here must "communicate" with each other; they all must sum to 1. This is a consequence of our network diagrams representing scalar values for each neuron rather than vector (or, more generally, **tensor**) values. More on alternative specifications of networks to come shortly.

Since we have multiple output neurons now, it makes more sense to represent the weights as a matrix rather than a vector, the same way that a matrix is required for the "statistical version" of the categorical regression model above.

The loss function most frequently used when training this type of network is called the **categorical cross-entropy** loss function $L(\mathbf{y}, \mathbf{y}^*) = -\mathbf{y}^T \log(\mathbf{y}^*)$. Note that the outputs and the true $y$-values of this model are vectors rather than scalars, so the loss function must take vector inputs. Like all loss functions, however, it must produce a scalar output.

## 2. Another Type of Diagram: Computational Graphs

In additional to the network diagrams above, **computational graphs** are also used to represent the same models. Whereas you might see the above diagrams in academic papers, computational graphs tend to convey the same information in a more condensed way. Here's what a logistic regression looks like as a computational graph:

![Logistic Regression Computational Graph with Loss](../pics/Expanded-Logistic-Regression-Computational-Graph2.png)

Unlike the previous nework diagrams, this shows the logistic regression when applied across all the inputs in $\mathbf{X}$. This explicitly shows the parameters as well as the computations between the various components of the model. *Arrows here do not correspond to weights*; the weights are already accounted for. The logistic function $\sigma(\cdot)$ is understood to be vectorized across its arguments. Also, you may notice that in general, the shapes of $\mathbf{Xw}$ and the scalar $b$ won't match. The packages that will be discussed later in this guide automatically implement **broadcasting** for operations with mismatched shapes according to [these rules](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html).

Also unlike the previous diagrams, the binary cross-entropy loss function, abbreviated here as BCE, is represented in the graph. Its input is the true $y$-values and the output from the sigmoid function. Its output is the value we are trying to minimize.

Note that with both the sigmoid and the loss function nodes, it's possible to further expand these to be more even more explicit about the exact operations that are being applied. Binary cross-entropy, for example, consists of multiplications, logarithms, and an averaging step, all of which could be given their own node in the graph.

Entities in computational graphs either represent tensors or operations on tensors. Tensors are generalizations of vectors (rank-1 tensors) or matrices (rank-2 tensors). A cube of numbers, for example, would be a rank-3 tensor. For an image processing task, a mini-batch of color images might be represented in a computational graph as a rank-4 tensor: 1 dimension indexing the samples, 2 spatial dimensions for the image, and a final dimension for the RGB color channels. We can imagine the tensors defined at the bottom of the graph moving along the edges and having operations applied to them. This notion is where the most popular neural network library [TensorFlow](https://www.tensorflow.org/) (discussed more later) gets its name.

### Autodifferentiation

As you can imagine when looking at the equations given in Section 1, we will frequently need the gradient of the average loss with respect to the parameters when training these networks. In simple cases, this is no problem, but this may be quite tedious to compute for larger models.

Luckily, modern neural network packages help us out by providing **autodifferentiation** functionality. This means that when we specify a neural network model using a certain API, we will be able to evaluate the gradient with *no manual calculus whatsoever*. Better yet, these are not finite difference derivatives; any numerical issues that occur have to do with how the computer represents numbers internally, but not because the approximation of the derivative is not precise enough.

## 3. Optimization

Statisticians frequently need to resort to numerical optimization methods when closed-form solutions don't exist for say, a log-likelihood maximization problem. Iterative optimization algorithms are extremely important when training neural networks, not only because of the intractability of the loss minimization problem but also because these methods may allow us to avoid having to keep all the data in memory at the same time. This can become very important considering the scale of the datasets that neural networks are frequently applied to.

Unlike statisticians, who frequently use optimization methods requiring second derivatives, those training neural networks almost exclusively use first-order methods. This, again, has to do with the scale of the problems that neural networks are used for; the networks may become so large that the Hessian matrix may not even fit into memory.

It should be noted that machine learners tend to phrase their optimization problems in terms of *minimizing* a function rather than maximizing it. This is because the metric being optimized tends to be a measure of loss rather than one of utility. Of course, minimizing a function is the same as maximizing its negative, and vice-versa.

### Batch Gradient Descent

Imagine we would like to minimize a function $\ell(\mathbf{\theta}; \mathbf{X}, \mathbf{y})$, where we would like to optimize over the parameter vector $\mathbf{\theta}$ and treat $\mathbf{X}$ as fixed. This is frequently something like a negative log-likelihood, as in the three cases mentioned above. Recall that the direction of steepest ascent of a function is its gradient $\nabla_\mathbf{\theta} \ell(\mathbf{\theta}; \mathbf{X}, \mathbf{y})$. If we view this function as a hillside we are standing on (at some current point $\mathbf{\theta}^{(i)}$), then in order to get to the bottom of the valley, we should move in the direction of the negative gradient. This leads to the following algorithm:
1. Initialize the value of $\mathbf{\theta}$ at some value $\mathbf{\theta}^{(0)}$
2. $\mathbf{\theta}^{(i + 1)} \leftarrow \mathbf{\theta}^{(i)} - \alpha \cdot \nabla_\mathbf{\theta} \ell(\mathbf{\theta}; \mathbf{X}, \mathbf{y}) \vert_{\mathbf{\theta} = \mathbf{\theta}^{(i)}}$
3. Repeat Step 2 until convergence

$\alpha$ here is called the **learning rate**. Values of $\alpha$ that are too high can lead to frequent overshooting of the true minimum, while values that are too low may make the algorithm take a long time to converge. When optimizing complex functions, the learning rate is often lowered from iteration or iteration or in response to some event. Therefore, it might be more accurate to write $\alpha$ above as $\alpha^{(i)}$.

In general, you will not actually minimize the loss function of a neural network! The loss functions being minimized are highly **non-convex** (intuitively, they don't look like bowls), and therefore gradient descent is prone to get stuck at local minima, have trouble with saddle points, etc. This problem has led to a lot of research into optimization methods for neural networks. With all this said, finding a good local minimum may be sufficient for the problem at hand. At the end of the day, performance on the test set is what matters, not where your optimization method ends up.

### Mini-batches

Consider the case where $\ell(\mathbf{\theta}; \mathbf{X}, \mathbf{y})$ is a negative log-likelihood, and suppose that the response values $y_i$ (which may be a vector in some cases) are independent. Then we can write our problem as:
$$ \underset{\mathbf{\theta}}{\operatorname{argmin}} \ell(\mathbf{\theta}; \mathbf{X}, \mathbf{y}) = \underset{\mathbf{\theta}}{\operatorname{argmin}} \frac{1}{n} \sum_{i = 1}^n \ell_i(\mathbf{\theta}; \mathbf{x}^{(i)}, y_i)$$
as the $\frac{1}{n}$ won't affect the result of the optimization. Because the joint distribution of the responses can be decomposed into the product of the marginals, the total loss here can be decomposed into the sum of indvidual per-datapoint loss terms.

Now let's think about the case where my entire dataset $\mathbf{X}$ doesn't fit in memory at once. In this case, it's useful to make the following approximation:
$$\nabla_\mathbf{\theta} \frac{1}{n} \sum_{i = 1}^n \ell_i(\mathbf{\theta}; \mathbf{x}^{(i)}, y_i) \approx \nabla_\mathbf{\theta} \frac{1}{m} \sum_{i^* = 1}^m \ell_{i^*}(\mathbf{\theta}; \mathbf{x}^{(i^*)}, y_{i^*})$$
where we are now averaging over a *randomly chosen subset* of our data points (that is $m < n$). A new subset is sampled on each iteration of the gradient descent algorithm. The averaging is necessary to keep everything on the same scale. This set of $m$ data points is called a **mini-batch**. Frequently, $\frac{n}{m}$ steps of the mini-batch gradient descent algorithm constitute one training **epoch**. This is how many steps would be necessary to see all of the data (though in general, some data points will be seen multiple times in an epoch, while others not at all).

The term **mini-batch** always implies multiple data points per batch. You may also encounter the term **stochastic gradient descent** (SGD), which will refer either to the algorithm just described or to the special case of it where $m = 1$.

Many alternatives to mini-batch gradient descent have been created, and you should almost never use the vanilla algorithm. A list of some popular alternatives, together with links to the original sources, can be found in the [Keras documentation](https://keras.io/optimizers/) (we'll be using Keras later).

### Backpropagation

For each iteration of our gradient descent algorithm, we imagine (a mini-batch worth of) $x$-values traveling from the input layer up to the output layer. The values computed along the way, either at the output layer or at intermediate **hidden layers**, are stored for use in the computation of the derivative. After this "forward pass", a "backward pass" occurs, where, starting from the loss function (which we can imagine lives just beyond the output layer) back to the input layer, the chain rule is applied in a computationally efficient way in order to take the derivatives with respect to the parameters. This is where the term **backpropagation** comes from, as well as the criticism that it's nothing more than the chain rule. As it turns out, naive methods of taking these derivatives will cripple the training of larger networks; the computational efficiency is key.

You will hear often neural network people talk about something like "the error signal propagating backward through the graph"; that is, information from the value of the loss function being used to tweak the parameter values via gradient descent. The notion of "gradient information" moving backward from output to input also motivates some of the heuristics used to train neural networks.

## 4. Implementations

All of the basic models from Section 1 can be trained in one line of code with the right package, but here I will take a slightly more roundabout way in order to demonstrate packages that people actually use for training neural networks more generally.

Python is the language most frequently used for neural networks. Though there are neural network packages in other languages, the vast majority of the research community tends to use those in Python, and they are accordingly the most developed. Increased computational horsepower (particularly with GPUs) is one of the main reasons why neural networks are experiencing their current boom in popularity after years of obscurity, and dedicated neural network packages let you take advantage of this without descending into the bowels of your computer seeking performance-enhancing boosts (that is, you don't need to write any CUDA C).

Probably the most popular neural network package is Google's [TensorFlow](https://www.tensorflow.org/), a C++ package with a highly-developed Python API (you don't need to know any C++ to use it). TensorFlow will help you specify computational graphs and run them. Many optimizations, including GPU connections, are done under the hood.

As it turns out, coding in TensorFlow is a bit difficult, requiring you to more or less manually specify the tensor transformations you want to apply (that is, the graph). Thankfully, living on top of TensorFlow is a higher-level API named [Keras](https://keras.io/). In my experience, Keras tends to operate at just the right level of abstraction: Every "chunk" of a neural network you might think of in your head translates to roughly one line of Keras code. When you train and evaluate your Keras model, however, you have the full power of TensorFlow working behind the scenes.

Keras can use other low-level neural network packages as backends, including [Theano](http://deeplearning.net/software/theano/) and Microsoft's [CNTK](https://www.microsoft.com/en-us/cognitive-toolkit/). It may be slightly advantageous to use these in some circumstances (such as Theano for training recurrent nets), but TensorFlow tends to do everything pretty well, so I will stick to that here.

### Loading the Data

This will load the data into 4 arrays:
- `X_train`, with shape `(60000, 28, 28)`
- `y_train`, with shape `(60000, 1)`
- `X_test`, with shape `(10000, 28, 28)`
- `y_test`, with shape `(10000, 1)`

The following function is originally taken from [this blog](https://martin-thoma.com/classify-mnist-with-pybrain/).

In [2]:
def get_labeled_data(imagefile, labelfile):
    images = gzip.open(imagefile, 'rb')
    labels = gzip.open(labelfile, 'rb')

    images.read(4)
    number_of_images = images.read(4)
    number_of_images = unpack('>I', number_of_images)[0]
    rows = images.read(4)
    rows = unpack('>I', rows)[0]
    cols = images.read(4)
    cols = unpack('>I', cols)[0]

    labels.read(4)
    N = labels.read(4)
    N = unpack('>I', N)[0]

    if number_of_images != N:
        raise Exception('number of labels did not match the number of images')

    x = np.zeros((N, rows, cols), dtype=np.float32)
    y = np.zeros((N, 1), dtype=np.uint8)
    for i in range(N):
        if i % 5000 == 0:
            print("Images loaded: %i" % i)
        for row in range(rows):
            for col in range(cols):
                tmp_pixel = images.read(1)
                tmp_pixel = unpack('>B', tmp_pixel)[0]
                x[i][row][col] = tmp_pixel
        tmp_label = labels.read(1)
        y[i] = unpack('>B', tmp_label)[0]
    return (x, y)

In [3]:
train_path = os.path.join('..', 'Data', 'fashion_mnist_train.npz')

if os.path.exists(train_path):
    train_file = np.load(train_path)
    X_train, y_train = tuple(train_file[arr] for arr in train_file.files)
else:
    X_train, y_train = get_labeled_data('../Data/train-images-idx3-ubyte.gz', '../Data/train-labels-idx1-ubyte.gz')
    np.savez(train_path, X=X_train, y=y_train)

In [4]:
test_path = os.path.join('..', 'Data', 'fashion_mnist_test.npz')

if os.path.exists(test_path):
    test_file = np.load(test_path)
    X_test, y_test = tuple(test_file[arr] for arr in test_file.files)
else:
    X_test, y_test = get_labeled_data('../Data/t10k-images-idx3-ubyte.gz', '../Data/t10k-labels-idx1-ubyte.gz')
    np.savez(test_path, X=X_test, y=y_test)

### Categorical Regression in Keras

The current shape of the input array `X_train` is (60000, 28, 28): 60000 28-by-28 grayscale images. For typical statistical models, we generally think of having a matrix as input rather than a rank-3 tensor, so first we reshape the data into a "flattened version" with shape (60000, 28 * 28) = (60000, 784). We do something similar for the test data.

In [5]:
X_train_flat = X_train.reshape(60000, 28 * 28)
X_test_flat = X_test.reshape(10000, 28 * 28)

Let's also center and scale the data to mitigate conditioning problems in our optimization.

In [6]:
scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

Remember that in terms of neural networks, a categorical regression consists of an input layer followed by a layer with as many neurons as there are classes. The activation function for the output layer is the softmax function.

In Keras, the most basic way to create a model is using its `Sequential` function. Unsurprisingly, this is used for networks which are a simple sequence of layers. In our case, we have the 10-neuron softmax output layer represented by the `Dense` function, and the input layer which is simply represented by the `input_shape` argument of `Dense`.

In [7]:
model = Sequential([
    Dense(10, input_shape=(784,), activation='softmax')
])

The next step is to **compile** the model. This combines the model with a loss function and an optimizer. Here I will use
`sparse_categorical_crossentropy`, which is the version of categorical cross-entropy you use when your categories are encoded as digits rather than one-hot vectors. The optimizer here will simply be `sgd` (for stochastic gradient descent, though we'll be able to use mini-batches) with the default learning rate.

In [8]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy')

Ok, now we need to fit the thing. Let's use a batch size of 500 with 100 epochs and see what we get.

In [9]:
model.fit(X_train_flat, y_train, batch_size=500, epochs=100, shuffle=True, verbose=0);

And now we get predictions and compare them to the true values:

In [10]:
y_pred = model.predict(X_test_flat, batch_size=500)
y_pred = np.argmax(y_pred, axis=1)

accuracy_score(y_test, y_pred)

0.84209999999999996

## 5. Networks with Hidden Layers

84% test set accuracy isn't horrible, but it's not great either, and is far from the best that we can achieve. The problem is with our features. Our current representation of the image is nothing more than the individual pixels that we were given; we have not yet made any effort to capture nonlinear interactions between the pixel values that may clue the algorithm in to the true class of the object. With better inputs, it is hoped, our categorical regression classifier should be able to get better accuracy. Good stuff in, good stuff out.

Traditional machine learning approaches require this feature creation to be done by hand. This works well for simple problems where we have a good intuition about what values may be good predictors of the class, but fails for complex problems like image classification. What we need is to *automate the feature creation process* or have our algorithms "learn" good **representations** of the data that will help our classifier do its job well.

This might us to an idea: Instead of passing in the raw pixel values to our classifier, we will instead pass (nonlinear) functions of the inputs, where the forms of these functions are controlled by additional parameter values that need to be learned. If the set of functions allowed is large enough, we hope, our training procedure should be able to pick parameter values corresponding to functions that capture useful values for our classifier.

So instead of calculating predicted values with:
$$f(\mathbf{x}^{(i)}) = \operatorname{softmax} \left( \mathbf{b} + \mathbf{W} \mathbf{x}^{(i)} \right)$$
we will instead use:
$$f(\mathbf{x}^{(i)}) = \operatorname{softmax} \left( \mathbf{b}^* + \mathbf{W}^* \mathbf{h}( \mathbf{x}^{(i)}; \mathbf{\theta} ) \right)$$
where $\mathbf{W}^*$ and $\mathbf{b}^*$ have the same basic meaning as $\mathbf{W}$ and $\mathbf{b}$, though the dimensionalities may be different. $\mathbf{h}( \cdot; \mathbf{\theta} )$ is a vector of (scalar-valued) functions that is used to transform the raw initial representation of the image $\mathbf{x}^{(i)}$ into a new set of features that then take their place, and hopefully improve the classification algorithm. Note that the number of functions in $\mathbf{h}( \cdot; \mathbf{\theta} )$ is not necessarily the same as the number of values in each $\mathbf{x}^{(i)}$. $\mathbf{\theta}$ represents new parameters that are used to control the shapes of these functions.

So what types of functions should we consider? The current standard is to use the same family of functions for each member of the $\mathbf{h}_{\mathbf{\theta}}(\cdot)$ vector: An affine tranformation followed by an activation function.
$$h_j(\mathbf{x}^{(i)}; \mathbf{w}_j^T, b_j) = a\left( b_j + \mathbf{w}_j^T \mathbf{x}^{(i)} \right)$$

Note that $\mathbf{w}_j$ and $b_j$ are different for different members of $\mathbf{h}(\cdot)$.

The activation function we will use here is the "rectified linear unit", or ReLU function:
$$\operatorname{relu}(\mathbf{x}) = \max(\mathbf{x}, \mathbf{0})$$
where the functions are considered to be vectorized across their arguments.

In [11]:
model = Sequential([
    Dense(50, input_shape=(784,), activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy')
model.fit(X_train_flat, y_train, batch_size=500, epochs=100, shuffle=True, verbose=1);

Epoch 1/100
60000/60000 [==============================] - 0s - loss: 1.1485     
Epoch 2/100
60000/60000 [==============================] - 0s - loss: 0.6995     
Epoch 3/100
60000/60000 [==============================] - 0s - loss: 0.6063     
Epoch 4/100
60000/60000 [==============================] - 0s - loss: 0.5567     
Epoch 5/100
60000/60000 [==============================] - 0s - loss: 0.5248     
Epoch 6/100
60000/60000 [==============================] - 0s - loss: 0.5017     
Epoch 7/100
60000/60000 [==============================] - 0s - loss: 0.4839     
Epoch 8/100
60000/60000 [==============================] - 0s - loss: 0.4699     
Epoch 9/100
60000/60000 [==============================] - 0s - loss: 0.4583     
Epoch 10/100
60000/60000 [==============================] - 0s - loss: 0.4485     
Epoch 11/100
60000/60000 [==============================] - 0s - loss: 0.4401     
Epoch 12/100
60000/60000 [==============================] - 0s - loss: 0.4327     
Epoch 13/100


60000/60000 [==============================] - 0s - loss: 0.2894     
Epoch 100/100
60000/60000 [==============================] - 0s - loss: 0.2887     


Now we get about 87% test error:

In [12]:
y_pred = model.predict(X_test_flat, batch_size=500)
y_pred = np.argmax(y_pred, axis=1)

accuracy_score(y_test, y_pred)

0.8679

What if we add yet another layer, so the inputs to the classifier themselves have nonlinear feature inputs?

In [13]:
model = Sequential([
    Dense(100, input_shape=(784,), activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy')
model.fit(X_train_flat, y_train, batch_size=500, epochs=100, shuffle=True, verbose=1);

Epoch 1/100
60000/60000 [==============================] - 1s - loss: 1.3548     
Epoch 2/100
60000/60000 [==============================] - 1s - loss: 0.7590     
Epoch 3/100
60000/60000 [==============================] - 1s - loss: 0.6329     
Epoch 4/100
60000/60000 [==============================] - 1s - loss: 0.5724     
Epoch 5/100
60000/60000 [==============================] - 1s - loss: 0.5342     
Epoch 6/100
60000/60000 [==============================] - 1s - loss: 0.5070     
Epoch 7/100
60000/60000 [==============================] - 1s - loss: 0.4863     
Epoch 8/100
60000/60000 [==============================] - 1s - loss: 0.4698     
Epoch 9/100
60000/60000 [==============================] - 1s - loss: 0.4561     
Epoch 10/100
60000/60000 [==============================] - 1s - loss: 0.4449     
Epoch 11/100
60000/60000 [==============================] - 1s - loss: 0.4350     
Epoch 12/100
60000/60000 [==============================] - 1s - loss: 0.4266     
Epoch 13/100


60000/60000 [==============================] - 1s - loss: 0.2523     
Epoch 100/100
60000/60000 [==============================] - 1s - loss: 0.2516     


The basic trend we see so far is more layers corresponds to more expressive functions, which in turn leads to better test error.

In [14]:
y_pred = model.predict(X_test_flat, batch_size=500)
y_pred = np.argmax(y_pred, axis=1)

accuracy_score(y_test, y_pred)

0.87270000000000003